In [30]:
import pandas as pd
import numpy as np

In [31]:
# helpful 'constants'

years = [
    "1992", "1993", "1994", "1995", "1996", "1997", "1998", "1999",
    "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007",
    "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015",
    "2016", "2017", "2018", "2019", "2020"
]

leagues = {
    "gb1": "english_premier_league",
    "gb2": "english_championship",
    "sp1": "spanish_primera_division",
    "gr1": "german_bundesliga_1",
    "fr1": "french_ligue_1",
    "it1": "italian_serie_a",
    "pr1": "portugese_liga_nos",
    "ru1": "russian_premier_liga",
    "nd1": "dutch_eredivisie"
}

inflation = {
    "1992": 2.17,
    "1993": 2.09,
    "1994": 2.05,
    "1995": 2.01,
    "1996": 1.94,
    "1997": 1.89,
    "1998": 1.84,
    "1999": 1.78,
    "2000": 1.75,
    "2001": 1.70,
    "2002": 1.67,
    "2003": 1.64,
    "2004": 1.59,
    "2005": 1.55,
    "2006": 1.51,
    "2007": 1.46,
    "2008": 1.40,
    "2009": 1.35,
    "2010": 1.35,
    "2011": 1.29,
    "2012": 1.23,
    "2013": 1.19,
    "2014": 1.16,
    "2015": 1.13,
    "2016": 1.12,
    "2017": 1.10,
    "2018": 1.06,
    "2019": 1.03,
    "2020": 1.00
}

In [32]:
# helpful functions

def path(year, league):
    """ generate a formatted path for the data
    """
    return f"../data/{year}/{leagues[league]}.csv"

def inflate(year, price):
    """ return price in 2020 dollars for GBP from a given year
    """
    return price * inflation[year]

In [33]:
# create main dataframe of all transfers from 1992-2020 by reading csv data

frames = []

for year in years:
    for league in leagues:
        temp = pd.read_csv(path(year, league))
        # only include paid permanet (free loans/free transfers not included)
        temp = temp[temp["fee_cleaned"] > 0]
        frames.append(temp)

transfers = pd.concat(frames, sort=False)

# drop records that have no window - If a club was relegated, its transfers are duplicated with a null transfer period
transfers = transfers[transfers["transfer_period"].isnull() != True].reset_index(drop=True)

In [34]:
transfers[transfers["player_name"] == "Stuart Ripley"]

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
2,Middlesbrough FC,Stuart Ripley,24.0,Midfielder,Blackburn,£1.80m,out,Summer,1.80,Premier League,1992,1992/1993
64,Blackburn Rovers,Stuart Ripley,24.0,Midfielder,Middlesbrough,£1.80m,in,Summer,1.80,Premier League,1992,1992/1993
2440,Southampton FC,Stuart Ripley,30.0,Midfielder,Blackburn,£2.03m,in,Summer,2.03,Premier League,1998,1998/1999
2452,Blackburn Rovers,Stuart Ripley,30.0,Midfielder,Southampton,£2.03m,out,Summer,2.03,Premier League,1998,1998/1999


In [35]:
# we want to ensure there are no duplicate transfers, which will be those transfers which are between teams in the leagues
# that this dataset covers. Thus we keep all incoming transfers and drop outgoing transfers if the player stays in the
# leagues within our dataset

# merge transfers that have the same player name so we can compare rows pairwise efficiently
merged = pd.merge(transfers.reset_index(), transfers, on='player_name')

# build our subsetting criteria, namely pairwise comparisons that are the same other than the
# transfer movement and clubs being reversed. Club names are actually different, so it is sufficient
# to just ensure that the season, fee, and period are equal
within_eu = (
             (merged["transfer_movement_x"] == "out") & 
             (merged["transfer_movement_y"] == "in") &
             (merged["transfer_period_x"] == merged["transfer_period_y"]) & 
             (merged["season_x"] == merged["season_y"]) &
             (merged["fee_cleaned_x"] == merged["fee_cleaned_y"])
            )

df_no_dups = ~transfers.index.isin(merged.loc[within_eu, 'index'].values)

transfers = transfers[df_no_dups]
transfers

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
0,Middlesbrough FC,Tommy Wright,26.0,Left Winger,Leicester,£819Th.,in,Summer,0.819,Premier League,1992,1992/1993
1,Middlesbrough FC,Jonathan Gittens,28.0,Defender,Southampton,£225Th.,in,Summer,0.225,Premier League,1992,1992/1993
3,Norwich City,Mark Robins,22.0,Centre-Forward,Man Utd,£900Th.,in,Summer,0.900,Premier League,1992,1992/1993
5,Norwich City,Paul Blades,27.0,Right-Back,Wolves,£340Th.,out,Summer,0.340,Premier League,1992,1992/1993
6,Norwich City,Adrian Pennock,21.0,Midfielder,Bournemouth,£32Th.,out,Summer,0.032,Premier League,1992,1992/1993
...,...,...,...,...,...,...,...,...,...,...,...,...
28388,Willem II Tilburg,Jorn Brondeel,26.0,Goalkeeper,Twente FC,£45Th.,in,Summer,0.045,Eredivisie,2020,2020/2021
28390,Ajax Amsterdam,Sébastien Haller,26.0,Centre-Forward,West Ham,£20.25m,in,Winter,20.250,Eredivisie,2020,2020/2021
28391,SC Heerenveen,Tibor Halilovic,25.0,Defensive Midfield,HNK Rijeka,£1.35m,in,Winter,1.350,Eredivisie,2020,2020/2021
28392,SC Heerenveen,Rami Kaib,23.0,Left-Back,Elfsborg,£450Th.,in,Winter,0.450,Eredivisie,2020,2020/2021


In [36]:
transfers[transfers["player_name"] == "Stuart Ripley"]

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
64,Blackburn Rovers,Stuart Ripley,24.0,Midfielder,Middlesbrough,£1.80m,in,Summer,1.80,Premier League,1992,1992/1993
2440,Southampton FC,Stuart Ripley,30.0,Midfielder,Blackburn,£2.03m,in,Summer,2.03,Premier League,1998,1998/1999


In [37]:
# add a column that specifies a simpler player position than those pre-populated
positions = pd.unique(transfers["position"])

attacker = ["Left Winger", "Second Striker", "Right Winger", "Centre-Forward", "Forward"]
midfielder = ["Attacking Midfield", "Defensive Midfield", "Midfielder", "Central Midfield", "Right Midfield", "Left Midfield"]
defender = ["Defender", "Left-Back", "Centre-Back", "Right-Back", "Sweeper"]
goalkeeper = ["Goalkeeper"]

simple_positions = []
for position in transfers["position"]:
    if position in attacker:
        simple_positions.append("attacker")
    elif position in midfielder:
        simple_positions.append("midfielder")
    elif position in defender:
        simple_positions.append("defender")
    elif position in goalkeeper:
        simple_positions.append("goalkeeper")
        
transfers = transfers.assign(position_simple=simple_positions)

In [38]:
transfers[transfers["player_name"] == "Adriano"]

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,position_simple
4658,FC Internazionale,Adriano,19.0,Centre-Forward,Flamengo,£11.87m,in,Summer,11.87,Serie A,2001,2001/2002,attacker
5173,AC Parma,Adriano,20.0,Centre-Forward,Inter,£13.05m,in,Summer,13.05,Serie A,2002,2002/2003,attacker
5667,FC Internazionale,Adriano,21.0,Centre-Forward,AC Parma,£21.06m,in,Winter,21.06,Serie A,2003,2003/2004,attacker
6077,Sevilla FC,Adriano,20.0,Left-Back,Coritiba FC,£1.89m,in,Winter,1.89,Primera Division,2004,2004/2005,defender
6444,CD Nacional,Adriano,25.0,Centre-Forward,Cruzeiro,£270Th.,out,Winter,0.27,Liga Nos,2004,2004/2005,attacker
8475,FC Porto,Adriano,27.0,Centre-Forward,Cruzeiro,£1.08m,in,Summer,1.08,Liga Nos,2006,2006/2007,attacker
9597,AS Monaco,Adriano,25.0,Right-Back,Atalanta,£2.25m,in,Summer,2.25,Ligue 1,2007,2007/2008,defender
13111,FC Barcelona,Adriano,25.0,Left-Back,Sevilla FC,£8.55m,in,Summer,8.55,Primera Division,2010,2010/2011,defender
21272,FC Barcelona,Adriano,31.0,Left-Back,Besiktas,£2.07m,out,Summer,2.07,Primera Division,2016,2016/2017,defender


In [39]:
# some players have the same name, so we should be careful about the visualization based on just name
# we will base an ID system off of position and age when a player has multiple transfers

players = transfers["player_name"].to_numpy()
positions = transfers["position"].to_numpy()
ages = transfers["age"].to_numpy()
year = transfers["year"].to_numpy()

player_id = np.zeros(len(players))
current_id = 0

combined = {}

# create a dictionary of combined indices, ages, transfer dates, and positions
# IDs given here will later be overwritten if a player is different
for idx, player in enumerate(players):
    if player in combined:
        combined[player].append([idx, ages[idx], year[idx], positions[idx]])
        # the index of the first occurence of the name can recall the ID used originally
        player_id[idx] = player_id[combined[player][0][0]]
    else:
        combined[player] = [[idx, ages[idx], year[idx], positions[idx]]]
        player_id[idx] = current_id
        current_id += 1

# # decision cases are when a player has multiple transfers and some are recorded with a different position
decision_cases = {}

for player in combined:
    if len(combined[player]) > 1:
        decision_cases[player] = [[combined[player][0]]]
        
        # base groups on positions, with the caveat that players who get younger in the same position make a new group
        # also ensure that ages are consistent across years and aren't off by more than one year (possible b/c of bdays 
        # inside of transfer windows)
        
        # essentially, the logic is that if a player is in the same position, older, and an age that makes sense given
        # the years since the last transaction, it is the same player.
        for record in combined[player][1:]:
            found = False
            for group in decision_cases[player]:
                if (
                   (group[-1][3] == record[3]) and 
                   (group[-1][1] <= record[1]) and 
                   abs((record[1] - group[-1][1]) - (record[2] - group[-1][2])) <= 1
                   ):
                    group.append(record)
                    found = True
                    
            if not found:
                decision_cases[player].append([record])

# now we handle these position groupings and give each group a unique ID
for player in decision_cases:
    for group in decision_cases[player]:
        for record in group:
            player_id[record[0]] = current_id
        current_id += 1
        
transfers = transfers.assign(player_id=player_id)

In [40]:
# Adriano is one of the more complex names in the dataset
transfers[transfers["player_name"] == "Adriano"]

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,position_simple,player_id
4658,FC Internazionale,Adriano,19.0,Centre-Forward,Flamengo,£11.87m,in,Summer,11.87,Serie A,2001,2001/2002,attacker,12077.0
5173,AC Parma,Adriano,20.0,Centre-Forward,Inter,£13.05m,in,Summer,13.05,Serie A,2002,2002/2003,attacker,12077.0
5667,FC Internazionale,Adriano,21.0,Centre-Forward,AC Parma,£21.06m,in,Winter,21.06,Serie A,2003,2003/2004,attacker,12077.0
6077,Sevilla FC,Adriano,20.0,Left-Back,Coritiba FC,£1.89m,in,Winter,1.89,Primera Division,2004,2004/2005,defender,12078.0
6444,CD Nacional,Adriano,25.0,Centre-Forward,Cruzeiro,£270Th.,out,Winter,0.27,Liga Nos,2004,2004/2005,attacker,12079.0
8475,FC Porto,Adriano,27.0,Centre-Forward,Cruzeiro,£1.08m,in,Summer,1.08,Liga Nos,2006,2006/2007,attacker,12079.0
9597,AS Monaco,Adriano,25.0,Right-Back,Atalanta,£2.25m,in,Summer,2.25,Ligue 1,2007,2007/2008,defender,12080.0
13111,FC Barcelona,Adriano,25.0,Left-Back,Sevilla FC,£8.55m,in,Summer,8.55,Primera Division,2010,2010/2011,defender,12078.0
21272,FC Barcelona,Adriano,31.0,Left-Back,Besiktas,£2.07m,out,Summer,2.07,Primera Division,2016,2016/2017,defender,12078.0


In [41]:
transfers[transfers["player_name"] == "Kevin De Bruyne"]

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,position_simple,player_id
14054,Chelsea FC,Kevin De Bruyne,20.0,Attacking Midfield,KRC Genk,£7.20m,in,Winter,7.200,Premier League,2011,2011/2012,midfielder,14206.0
15830,SV Werder Bremen,Kevin De Bruyne,21.0,Attacking Midfield,Chelsea,Loan fee:£405Th.,in,Summer,0.405,1 Bundesliga,2012,2012/2013,midfielder,14206.0
17253,VfL Wolfsburg,Kevin De Bruyne,22.0,Attacking Midfield,Chelsea,£19.80m,in,Winter,19.800,1 Bundesliga,2013,2013/2014,midfielder,14206.0
19351,Manchester City,Kevin De Bruyne,24.0,Attacking Midfield,VfL Wolfsburg,£68.40m,in,Summer,68.400,Premier League,2015,2015/2016,midfielder,14206.0


In [42]:
pd.to_csv("../data/cleaned_all_transf")
transfers

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,position_simple,player_id
0,Middlesbrough FC,Tommy Wright,26.0,Left Winger,Leicester,£819Th.,in,Summer,0.819,Premier League,1992,1992/1993,attacker,10934.0
1,Middlesbrough FC,Jonathan Gittens,28.0,Defender,Southampton,£225Th.,in,Summer,0.225,Premier League,1992,1992/1993,defender,1.0
3,Norwich City,Mark Robins,22.0,Centre-Forward,Man Utd,£900Th.,in,Summer,0.900,Premier League,1992,1992/1993,attacker,10937.0
5,Norwich City,Paul Blades,27.0,Right-Back,Wolves,£340Th.,out,Summer,0.340,Premier League,1992,1992/1993,defender,3.0
6,Norwich City,Adrian Pennock,21.0,Midfielder,Bournemouth,£32Th.,out,Summer,0.032,Premier League,1992,1992/1993,midfielder,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28388,Willem II Tilburg,Jorn Brondeel,26.0,Goalkeeper,Twente FC,£45Th.,in,Summer,0.045,Eredivisie,2020,2020/2021,goalkeeper,15876.0
28390,Ajax Amsterdam,Sébastien Haller,26.0,Centre-Forward,West Ham,£20.25m,in,Winter,20.250,Eredivisie,2020,2020/2021,attacker,15430.0
28391,SC Heerenveen,Tibor Halilovic,25.0,Defensive Midfield,HNK Rijeka,£1.35m,in,Winter,1.350,Eredivisie,2020,2020/2021,midfielder,10931.0
28392,SC Heerenveen,Rami Kaib,23.0,Left-Back,Elfsborg,£450Th.,in,Winter,0.450,Eredivisie,2020,2020/2021,defender,10932.0
